In [1]:
import os

os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/ra_joint_predictions')

In [2]:
from utils.config import Config

config = Config()

In [3]:
from dataset.joint_val_dataset import hands_wrists_val_dataset

wrists_ds = hands_wrists_val_dataset(config, repeat = False)

wrist_j_data = wrists_ds.create_wrists_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = False)
wrist_e_data = wrists_ds.create_wrists_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = True)

2020-06-27 11:46:39,900;WARNING - Missing permissions to create directory for caching!
2020-06-27 11:46:39,904;WARNING - Missing permissions to create directory for caching!
[{'augment': <function random_brightness_and_contrast at 0x7fb98e0de268>}, {'augment': <function random_crop at 0x7fb98e0de378>, 'params': {'min_scale': 0.9}}, {'augment': <function random_gaussian_noise at 0x7fb98e0de400>, 'p': 0.2}, {'augment': <function random_rotation at 0x7fb98e0de2f0>, 'params': {'max_rot': 10}}]
2020-06-27 11:46:44,629;WARNING - Missing permissions to create directory for caching!
2020-06-27 11:46:44,632;WARNING - Missing permissions to create directory for caching!
[{'augment': <function random_brightness_and_contrast at 0x7fb98e0de268>}, {'augment': <function random_crop at 0x7fb98e0de378>, 'params': {'min_scale': 0.9}}, {'augment': <function random_gaussian_noise at 0x7fb98e0de400>, 'p': 0.2}, {'augment': <function random_rotation at 0x7fb98e0de2f0>, 'params': {'max_rot': 10}}]


In [4]:
import numpy as np

from tensorflow import keras
from model.utils.metrics import mae_metric, rmse_metric, class_filter_rmse_metric, softmax_mae_metric, softmax_rmse_metric, class_filter_softmax_rmse_metric

def eval_model(label, dataset, model_paths, max_output, rounding_cutoff = 0.3):
    truths = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: []
    }
    
    preds = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: []
    }
    
    models = [keras.models.load_model(model_path, compile = False) for model_path in model_paths]
    
    for x, y in dataset:
        y_preds = np.zeros((x.shape[0], 6))
        
        for model in models:
            y_pred = model.predict(x)
            
            for n in range(6):
                y_preds[:, n] += y_pred[n][:, 0]
                
        y_preds = y_preds / len(models)
        
        for n in range(6):
            truths[n].extend(y[n][:, 0].numpy())
            preds[n].extend(y_preds[:, n])
    
    loss = np.zeros(6)
    rmse = np.zeros(6)
    mae = np.zeros(6)
    filter_rmse = np.zeros(6)
            
    for idx in range(6):
        t_vals = np.array(truths[idx])
        p_vals = np.array(preds[idx])
        
        p_vals[p_vals < 0] = 0
        p_vals[p_vals > max_output] = max_output
        p_vals[p_vals <= rounding_cutoff] = 0.0
        
        non0_idx = np.where(t_vals != 0.0)[0]
        
        loss[idx] = np.mean(np.square(t_vals - p_vals))
        rmse[idx] = np.sqrt(np.mean(np.square(t_vals - p_vals)))
        filter_rmse[idx] = np.sqrt(np.mean(np.square(t_vals[non0_idx] - p_vals[non0_idx])))
        mae[idx] = np.mean(np.absolute(t_vals - p_vals))
    
    print('Model:', label)
    print('Loss:', loss)
    print('MAE:', mae)
    print('RMSE:', rmse)
    print('Filter RMSE:', filter_rmse)
    

In [5]:
from prediction.joint_damage_prediction import _robust_mean, joint_damage_predictor, joint_damage_type_predictor, wrist_pred_augments
import dataset.ops.image_ops as img_ops

from dataset.ops.dataset_ops import _augment_and_clip_image

class changed_ensembled_filter_predictor():
    def __init__(self, model_parameters, model_base_path = '../resources', aggregator = _robust_mean, rounding_cutoff = 0, no_augments = 50):
        self.model_parameters = model_parameters

        self.no_outcomes = model_parameters['no_outcomes']
        self.is_wrist = model_parameters.get('is_wrist', False)

        self.no_pred_models = model_parameters.get('no_pred_models', 0)
        self.no_filter_models = model_parameters.get('no_filter_models', 0)
        
        self.filter_cutoff = self.model_parameters.get('damage_type_cutoff', 0.2)
        # Value to return if the filter_prediction exceeds the cutoff
        self.default_value = self.model_parameters.get('default_value', 0.0)
        
        self.aggregator = _robust_mean
        self.rounding_cutoff = rounding_cutoff
        self.no_augments = no_augments
        
        self.augments = []

        self.n_filtered_images = 0
        self.n_processed_images = 0
        self.model_base_path = model_base_path

        self._init_model()

    def _init_model(self):
        def _get_predictor(model_index, pred_type):
            predictor = pred_type(self.model_parameters, self.model_base_path, model_index = model_index)
            
            return predictor
        
        self.damage_predictors = list(map(lambda no_model: _get_predictor(no_model, joint_damage_predictor), range(self.no_pred_models)))
        self.filter_predictors = list(map(lambda no_model: _get_predictor(no_model, joint_damage_type_predictor), range(self.no_filter_models)))

        if self.is_wrist:
            self.augments = img_ops.create_augments(wrist_pred_augments)
        else:
            self.augments = img_ops.create_augments(joint_pred_augments)
    
    def predict_joint_damage(self, img):
        self.n_processed_images += self.no_outcomes

        no_preds = self.no_augments + 1
        
        dmg_preds = np.zeros((self.no_pred_models, no_preds, self.no_outcomes))
        filter_preds = np.zeros((self.no_filter_models, no_preds, self.no_outcomes))

        # dmg_preds = np.zeros((len(self.damage_predictors * no_preds), self.no_outcomes))
        # filter_preds = np.zeros((len(self.filter_predictors * no_preds), self.no_outcomes))
        
        dmg_idx = 0
        filter_idx = 0

        for n in range(self.no_augments + 1):
            if n > 0:
                pred_img, _ = _augment_and_clip_image(img, [], augments = self.augments)
            else:
                pred_img = img

            for idx, damage_predictor in enumerate(self.damage_predictors):
                dmg_preds[idx, n, :] = damage_predictor.predict_joint_damage(pred_img)
                # dmg_idx += 1
            
            for idx, filter_predictor in enumerate(self.filter_predictors):
                filter_preds[idx, n, :] = filter_predictor.predict_joint_damage(pred_img)
                # filter_idx += 1
        
        full_dmg_preds = np.zeros(self.no_outcomes)
        for n in range(self.no_pred_models):
            model_dmg_preds, _ = self.aggregator(dmg_preds[n, :, :], 0)
            
            full_dmg_preds += model_dmg_preds
        full_dmg_preds = full_dmg_preds / self.no_pred_models
        
        full_dmg_preds[full_dmg_preds <= self.rounding_cutoff] = 0.0
        dmg_pred_labels = []
              
        if self.no_filter_models > 0:
            filter_preds, _ = self.aggregator(filter_preds)
        
            filtered_idx = np.where(filter_preds <= self.filter_cutoff)[0]
        
            dmg_preds[filtered_idx] = self.default_value
            for idx in filtered_idx:
                dmg_pred_labels[idx] = 'F'

            self.n_filtered_images += filtered_idx.size
        
        return full_dmg_preds, dmg_pred_labels

In [6]:
new_wrist_j_paths = ['../trained_models/narrowing/v8/wrists_narrowing_small_bottlenecked_dense_1M_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_right_group_test.h5',
'../trained_models/narrowing/v8/wrists_narrowing_small_bottlenecked_dense_1M_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_retrain_test.h5',
'../trained_models/narrowing/v8/wrists_narrowing_small_bottlenecked_dense_1M_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_left_group_test.h5']

eval_model('Wrists Narrowing (Small Dense)', wrist_j_data[1], new_wrist_j_paths, 4)

Model: Wrists Narrowing (Small Dense)
Loss: [0.25806531 0.42054583 0.2225912  0.48617979 0.31005071 0.22732038]
MAE: [0.23022227 0.3234751  0.16860949 0.36811691 0.19519952 0.16282155]
RMSE: [0.50800129 0.64849505 0.47179572 0.69726594 0.55682197 0.47678127]
Filter RMSE: [0.96397454 1.12498698 1.09410753 1.109662   1.45896366 1.25822784]


In [9]:
new_gap_wrist_j_paths = ['../trained_models/narrowing/v8/wrists_narrowing_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_test.h5',
'../trained_models/narrowing/v8/wrists_narrowing_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_right_group_test.h5',
'../trained_models/narrowing/v8/wrists_narrowing_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_left_group_test.h5']

eval_model('Wrists Narrowing (Complex Gap)', wrist_j_data[1], new_gap_wrist_j_paths, 4)

Model: Wrists Narrowing (Complex Gap)
Loss: [0.28875939 0.43108322 0.29609063 0.47099617 0.36327482 0.3216108 ]
MAE: [0.25640581 0.33301869 0.23363415 0.38499197 0.23146999 0.20908823]
RMSE: [0.53736337 0.65656928 0.5441421  0.68629161 0.60272284 0.5671074 ]
Filter RMSE: [0.97773874 1.11043775 1.24488462 1.0747818  1.5552891  1.48716337]


In [10]:
new_ensemble_wrist_j_paths = ['../trained_models/narrowing/v8/wrists_narrowing_small_bottlenecked_dense_1M_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_right_group_test.h5',
'../trained_models/narrowing/v8/wrists_narrowing_small_bottlenecked_dense_1M_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_retrain_test.h5',
'../trained_models/narrowing/v8/wrists_narrowing_small_bottlenecked_dense_1M_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_left_group_test.h5',
'../trained_models/narrowing/v8/wrists_narrowing_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_test.h5',
'../trained_models/narrowing/v8/wrists_narrowing_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_right_group_test.h5',
'../trained_models/narrowing/v8/wrists_narrowing_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_left_group_test.h5']

eval_model('Wrists Narrowing (Ensemble)', wrist_j_data[1], new_ensemble_wrist_j_paths, 4)

Model: Wrists Narrowing (Ensemble)
Loss: [0.26405111 0.40854151 0.2394563  0.46792689 0.32167802 0.26195519]
MAE: [0.23643628 0.32600655 0.19996346 0.36773868 0.21084674 0.18259521]
RMSE: [0.51385904 0.63917252 0.48934272 0.68405182 0.56716666 0.51181558]
Filter RMSE: [0.95744813 1.096001   1.12406164 1.08324834 1.47717528 1.34967298]


In [7]:
new_wrist_e_paths = ['../trained_models/erosion/v8/wrists_erosion_small_bottlenecked_dense_1M_with_fc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_100frozen_030maj_fixed_bs_retrain_test.h5',
'../trained_models/erosion/v8/wrists_erosion_small_bottlenecked_dense_1M_with_fc_192x256_64bs_60steps_100epochs_adamW_1e3_1e6_200frozen_nodecay_030maj_fixed_bs_left_group_test.h5',
'../trained_models/erosion/v8/wrists_erosion_small_bottlenecked_dense_1M_with_fc_192x256_64bs_60steps_100epochs_adamW_1e3_1e6_200frozen_nodecay_030maj_fixed_bs_right_group_test.h5']

eval_model('Wrists Erosion (Small Dense)', wrist_e_data[1], new_wrist_e_paths, 5)

Model: Wrists Erosion (Small Dense)
Loss: [0.62691021 0.56178871 0.89151958 0.6236578  0.77611893 1.01924282]
MAE: [0.27203505 0.35950415 0.32937882 0.44335619 0.29855391 0.39740948]
RMSE: [0.79177662 0.74952566 0.94420314 0.78972007 0.88097612 1.00957556]
Filter RMSE: [2.14906771 1.56752761 2.5265665  1.31812526 2.43593932 2.11694825]


In [11]:
new_gap_wrist_e_paths = ['../trained_models/erosion/v8/wrists_erosion_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_test.h5',
'../trained_models/erosion/v8/wrists_erosion_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_right_group_test.h5',
'../trained_models/erosion/v8/wrists_erosion_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_left_group_test.h5']

eval_model('Wrists Erosion (Gap)', wrist_e_data[1], new_gap_wrist_e_paths, 5)

Model: Wrists Erosion (Gap)
Loss: [0.78329857 0.58327718 0.82668157 0.63006312 0.76763671 0.95308672]
MAE: [0.29407076 0.35283409 0.31776214 0.47775866 0.28995074 0.41296563]
RMSE: [0.88504157 0.76372586 0.90922031 0.79376515 0.8761488  0.9762616 ]
Filter RMSE: [2.41223593 1.61681981 2.37802422 1.37178929 2.4126951  2.10124119]


In [12]:
new_ensemble_wrist_e_paths = ['../trained_models/erosion/v8/wrists_erosion_small_bottlenecked_dense_1M_with_fc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_100frozen_030maj_fixed_bs_retrain_test.h5',
    '../trained_models/erosion/v8/wrists_erosion_small_bottlenecked_dense_1M_with_fc_192x256_64bs_60steps_100epochs_adamW_1e3_1e6_200frozen_nodecay_030maj_fixed_bs_left_group_test.h5',
    '../trained_models/erosion/v8/wrists_erosion_small_bottlenecked_dense_1M_with_fc_192x256_64bs_60steps_100epochs_adamW_1e3_1e6_200frozen_nodecay_030maj_fixed_bs_right_group_test.h5',
    '../trained_models/erosion/v8/wrists_erosion_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_test.h5',
    '../trained_models/erosion/v8/wrists_erosion_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_right_group_test.h5',
    '../trained_models/erosion/v8/wrists_erosion_complex_gap_nonfc_192x256_64bs_60steps_200epochs_adamW_1e3_1e6_50frozen_nodecay_030maj_fixed_bs_left_group_test.h5']

eval_model('Wrists Erosion (Ensemble)', wrist_e_data[1], new_ensemble_wrist_e_paths, 5)

Model: Wrists Erosion (Ensemble)
Loss: [0.68242849 0.56649769 0.84299965 0.59549258 0.77499223 0.96392095]
MAE: [0.27615426 0.35844626 0.3207504  0.45120074 0.28863237 0.40373243]
RMSE: [0.8260923  0.75266041 0.91815012 0.77168165 0.88033643 0.98179476]
Filter RMSE: [2.25098889 1.58884357 2.43191108 1.31822249 2.43648846 2.09013353]
